In [115]:
import os, json, glob
import pandas as pd
from pandas import ExcelFile
import numpy as np
from passyunk.parser import PassyunkParser

C:\Users\catst\Anaconda2\lib\site-packages\passyunk\parser.py:619: UserWarning: USPS file not found.
  warnings.warn('USPS file not found.')
C:\Users\catst\Anaconda2\lib\site-packages\passyunk\parser.py:625: UserWarning: Election file not found.
  warnings.warn('Election file not found.')


In [33]:
with open("cols.json") as cols_file:
    column_names = json.load(cols_file)['cols']
print len(column_names)

43


In [25]:
cols = ['ID','last_name','first_name','gender','house_number','street','apartment_number','zip','council_district']
f = 'Philly Voter List 3.xlsx'
to_str = lambda x: str(x)
df = pd.read_excel(f,header=None, nrows=10,names=cols, usecols=[0,2,3,6,12,14,15,19,38],converters={'house_number':to_str,'zip':to_str})

In [23]:
df['short_address'] = df['house_number'].astype(str) + df['street']

In [29]:
df['house_number']

0          526.0
1         1702.0
2         1121.0
3          723.0
4         1401.0
5          250.0
6          145.0
7         1003.0
8          222.0
9          709.0
10         644.0
11         816.0
12         437.0
13         615.0
14        1433.0
15         814.0
16        1512.0
17         839.0
18         236.0
19        1311.0
20        1533.0
21         704.0
22        1611.0
23        1524.0
24        1415.0
25         434.0
26        1413.0
27         507.0
28         917.0
29         430.0
          ...   
49970      425.0
49971    10204.0
49972     2701.0
49973    12407.0
49974     3870.0
49975    10733.0
49976    12209.0
49977     3750.0
49978     3750.0
49979    12209.0
49980     9923.0
49981    11921.0
49982    10923.0
49983     4165.0
49984     3721.0
49985     3517.0
49986     3501.0
49987     9951.0
49988     3750.0
49989      189.0
49990    10922.0
49991     2958.0
49992    11843.0
49993     4257.0
49994    11713.0
49995     3643.0
49996    12242.0
49997     3400

In [36]:
allFiles = glob.glob("Philly*.xlsx")
frame = pd.DataFrame()
list_ = []
#cols = ['first_name','last_name','gender','council_district','house_number','street','zip']
for file_ in allFiles:
    df = pd.read_excel(file_,index_col=0, header=None, names=column_names, usecols=range(0,44))
    list_.append(df)
frame = pd.concat(list_)

In [64]:
fairhill = frame.loc[frame['zip']==19133]

fairhill.to_csv('fairhill.csv',encoding='utf-8')

In [71]:
properties = pd.read_csv('../opa_properties_public.csv',usecols=['location','owner_1','owner_2','lat','lng'])

In [74]:
fairhill['location'] = fairhill['house_number'].astype(str) + " " + fairhill['street']

C:\Users\catst\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [131]:
p = PassyunkParser()
fairhill['location2'] = fairhill['location'].apply(lambda x: p.parse(x)['components']['output_address'])

C:\Users\catst\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [132]:
properties['location2'] = properties['location'].apply(lambda x: p.parse(x)['components']['output_address'])

<h1> FUCK IT </h1>

In [148]:
geocoded_voters_standardized = pd.merge(fairhill,properties,how='left',on='location2')

In [149]:
geocoded_voters_standardized

,last_name,first_name,gender,house_number,street,apartment_number,zip,council_district,location_x,location2,ID,location_y,owner_1,owner_2,lat,lng
0,GARCIA,JAZMINE,F,158,W ALLEGHENY AVE,NaN,19133,CTC07,158 W ALLEGHENY AVE,158 W ALLEGHENY AVE,015925836-51,158 W ALLEGHENY AVE,RIVERA ULISES JR,NaN,39.998640,-75.131995
1,OLMEDO,ELIUD,U,2839,N HOWARD ST,NaN,19133,CTC07,2839 N HOWARD ST,2839 N HOWARD ST,015926733-51,2839 N HOWARD ST,BELTRAN CECILIO,NaN,39.993485,-75.131227
2,MARBLEY,ANDRE,M,174,W ALLEGHENY AVE,38,19133,CTC07,174 W ALLEGHENY AVE,174 W ALLEGHENY AVE,015936329-51,NaN,NaN,NaN,NaN,NaN
3,DAVENPORT,LENNARD,M,123,W WISHART ST,NaN,19133,CTC07,123 W WISHART ST,123 W WISHART ST,015938055-51,123 W WISHART ST,ALSTON ANITA,NaN,39.998233,-75.130042
4,AGOSTO,LIZZETTE,F,2727,N MASCHER ST,NaN,19133,CTC07,2727 MASCHER ST,2727 MASCHER ST,015939879-51,2727 MASCHER ST,SANTIAGO FERNANDO,NaN,39.991832,-75.132559
5,ARROYO,CHRISTINE,F,2906,N MUTTER ST,NaN,19133,CTC07,2906 N MUTTER ST,2906 N MUTTER ST,015944456-51,NaN,NaN,NaN,NaN,NaN
6,GONZALEZ,ROSA,F,2727,N HOPE ST,NaN,19133,CTC07,2727 HOPE ST,2727 HOPE ST,015944718-51,2727 HOPE ST,PEREZ CONNIE TR,NaN,39.991798,-75.131129
7,CINTRON,ALEXANDRIA,F,2754,N HOWARD ST,NaN,19133,CTC07,2754 N HOWARD ST,2754 N HOWARD ST,015945373-51,2754 N HOWARD ST,ELMER MONTANEZ,ALEJ,39.992240,-75.131875
8,TORRES,LESLIE,F,2724,N WATERLOO ST,NaN,19133,CTC07,2724 N WATERLOO ST,2724 N WATERLOO ST,015945943-51,NaN,NaN,NaN,NaN,NaN
9,SANTIAGO,LUIS,U,3101,N PALETHORP ST,NaN,19133,CTC07,3101 PALETHORP ST,3101 PALETHORP ST,015946390-51,3101 PALETHORP ST,RODRIQUEZ DIANA,NaN,39.997461,-75.132851


In [138]:
len(geocoded_voters_standardized.loc[geocoded_voters_standardized['lat'].isnull()])

3981

In [150]:
geocoded_voters_standardized  = geocoded_voters_standardized.drop(labels=['location2','location_y'],axis=1)

In [152]:
output = geocoded_voters_standardized.loc[~geocoded_voters_standardized['lat'].isnull()]

In [157]:
output = output.rename(columns={'location_x':'short_address'})

In [160]:
output.to_csv('fairhill.csv',encoding='utf-8')

In [101]:
properties.loc[properties['location'].str.contains('02ND ST')].head()

,location,owner_1,owner_2,lat,lng
498805,151-59 N 02ND ST,SPIVAK ADAM,NaN,39.953422,-75.142865
498909,151-59 N 02ND ST,CHABRA NARINDER,TRUSTEE 13 S BANK TRUST,39.953390,-75.142499
498910,151-59 N 02ND ST,FIREMANS PLACE OWNERS ASS,NaN,39.953350,-75.142694
498911,151R-59 N 02ND ST,159 NORTH 2ND ST ASSOC,LLC,39.953497,-75.142600
498912,151R-59 N 02ND ST,159 NORHT 2ND ST ASSOC,LLC,39.953492,-75.142553


In [106]:
geocoded_voters['short_street'] = geocoded_voters.loc[geocoded_voters['lat'].isnull()]['street'].str[2:]
geocoded_voters = geocoded_voters.rename({'location':'location1'})
geocoded_voters['location'] = geocoded_voters['house_number'].astype(str) + " " + geocoded_voters['short_street']

In [108]:
geocode_2 = pd.merge(geocoded_voters,properties,on='location',how='left')

In [111]:
len(geocoded_voters.loc[geocode_2['lat_x'].isnull() & geocode_2['lat_y'].isnull()])

7772

In [112]:
len(fairhill)

18778

In [89]:
len(properties.loc[properties['lat'].isnull()]) / float(len(properties))

0.000854697909091849

In [98]:
frame.loc[frame['street'].str.contains('MASCHER ST',na=False)]

,last_name,first_name,gender,house_number,street,apartment_number,zip,council_district
0,,,,,,,,
016070349-51,RICHMOND,CAROLYNN,F,53,N MASCHER ST,3R,19106,CTC01
015938851-51,RIVERA,GABRIEL,M,3330,N MASCHER ST,NaN,19140,CTC07
015939879-51,AGOSTO,LIZZETTE,F,2727,N MASCHER ST,NaN,19133,CTC07
015961819-51,MORALES,ADELAIDA,F,3404,N MASCHER ST,NaN,19140,CTC07
015965336-51,RODRIGUEZ,DAVID,M,3332,N MASCHER ST,NaN,19140,CTC07
015981815-51,TOLEDO,ANGELO,M,2942,N MASCHER ST,NaN,19133,CTC07
015984082-51,ACEVEDO,EUSTACIO,M,3450,N MASCHER ST,NaN,19140,CTC07
016013985-51,DAVILA AGOSTO,CARMEN,U,2749,N MASCHER ST,NaN,19133,CTC07
016070271-51,BETANCOURT,YESENIA,F,3222,N MASCHER ST,NaN,19140,CTC07


In [55]:
frame = frame[['unknown1','last_name','suffix','affiliation','unknown2','street','state','cg']]

In [57]:
frame.columns = ['last_name','first_name','gender','house_number','street','apartment_number','zip','council_district']

In [34]:
focus_districts = frame[(frame['council_district'] == "CTC03") | (frame['council_district'] == "CTC08") | (frame['council_district'] == "CTC09")]

focus_districts.to_json('focus_districts.json',orient="records")

focus_districts = focus_districts.set_index('ID')

len(focus_districts)

322125

In [35]:
evictions = pd.read_csv("merged_1990.csv")

C:\Users\catst\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
evictions.head()

,Unnamed: 0,id,premises_x,lat,lon,number,street,city,state,zip,...,jba_breach,non_residential,unaware,months,publichousing,first_hearing_dow,latitude,longitude,clean_address,firstname
0,0,LT-06-11-01-3209,"6901 OLD YORK RD, PHILADELPHIA, PA",40.058737,-75.138612,6901,Old York Rd,Philadelphia,PA,19126,...,f,f,f,1.0,f,1.0,NaN,NaN,NaN,NaN
1,1,LT-94-06-16-1577,"220 W LUZERNE ST, PHILADELPHIA, PA",40.011256,-75.130653,220,W Luzerne St,Philadelphia,PA,19140,...,f,f,f,NaN,f,4.0,NaN,NaN,NaN,NaN
2,2,LT-06-12-29-7247,"6038 OGONTZ AVE, PHILADELPHIA, PA",40.047536,-75.151097,6038,Ogontz Ave,Philadelphia,PA,19141,...,f,f,f,0.0,f,2.0,NaN,NaN,NaN,NaN
3,4,LT-94-09-16-1407,"128 S 56TH ST, PHILADELPHIA, PA",39.957997,-75.233559,128,S 56th St,Philadelphia,PA,19139,...,f,f,f,NaN,f,2.0,NaN,NaN,NaN,NaN
4,5,LT-94-12-21-1995,"1516 JUDSON WAY, PHILADELPHIA, PA",39.978381,-75.173388,1516,Judson Way,Philadelphia,PA,19121,...,f,f,f,NaN,f,3.0,NaN,NaN,NaN,NaN


In [100]:
evictions['defendant'] = evictions['defendant'].str.upper()
evictions['voter_record'] = ""

In [104]:
focus_districts.iloc[0]

unknown1                                      NaN
last_name                                    HILL
first_name                                      R
middle_initial                                  C
suffix                                        NaN
gender                                          M
birth_date                    1972-04-23 00:00:00
date_registered               1997-03-17 00:00:00
affiliated                                      I
affilitation_date             2017-01-19 00:00:00
affiliation                                     D
house_number                                 5922
unknown2                                      NaN
street                                  IRVING ST
apartment_number                              NaN
address2                                      NaN
city                                 PHILADELPHIA
state                                          PA
zip                                         19139
unk3                                          NaN


In [105]:
for idx, voter in focus_districts.sample(frac=.01).iterrows():
    first = voter['first_name']
    last = voter['last_name']
    voter_id = idx
    mask = (evictions['defendant'].str.contains(first)) & (evictions['defendant'].str.contains(last))
    evictions['voter_record'] = evictions['voter_record'].mask(mask,voter_id)

KeyboardInterrupt: 

In [108]:
matched = evictions.loc[evictions['voter_record'] != ""]

In [128]:
matched.to_json('evicted_voters.json',orient="records")

In [124]:
focus_districts.loc[focus_districts.index == '015794249-51'][['first_name', 'last_name','house_number','street','zip']]

,first_name,last_name,house_number,street,zip
ID,,,,,
015794249-51,GENEEN,BIBBS,6000,N 13TH ST,19141


In [126]:
matched[['premises_x','zip','defendant','voter_record']]

,premises_x,zip,defendant,voter_record
182,"1903 S JESSUP ST, PHILADELPHIA, PA",19148,CHRISTOPHER JONES AND ALL OCCUPANTS|AND|GERALD...,015753706-51
370,"5051 OXFORD AVE, PHILADELPHIA, PA",19124,KEITH M. JOHNSON & OCCUPANTS,015717129-51
678,"3449 N HOPE ST, PHILADELPHIA, PA",19140,LEONARD WILLIAMS AND ALL OCCS|AND|MIESHA WRIGH...,015688308-51
712,"824 E HILTON ST, PHILADELPHIA, PA",19134,ERICK JOHNSON|AND|TERRENCE WILLIAMS,015031992-51
749,"4501 LARCHWOOD AVE, PHILADELPHIA, PA",19143,WILLIAMSON WALLACE,104409161-51
894,"1519 LATONA ST, PHILADELPHIA, PA",19146,DAVID WILLIAMS,021870087-51
936,"5000 WOODBINE AVE, PHILADELPHIA, PA",19131,BROWN REBECCA M,014596668-51
964,"2979 W SCHOOLHOUSE LN, PHILADELPHIA, PA",19144,KAREN BYRD,015913397-51
1329,"1100 S 58TH ST, PHILADELPHIA, PA",19143,HILLIAN BARBARA|AND|HILLIAN NAKIA,014593082-51
1475,"2711 TITAN ST, PHILADELPHIA, PA",19146,CHARLES POWELL|AND|LAVERNE HALL,104452652-51


In [ ]:
matched_voter_ids = matched['voter_record']
focus_districts.iloc

In [48]:
defendant = evictions.iloc[12]['defendant'].upper()
voter = focus_districts.iloc[0]
'ARTHUR' in defendant and 'MCDANIELS' in defendant

True

In [66]:
test = evictions.sample(frac=.01)
test['voter_record'] = ""
#for i, record in test:
    #test.iat[i,91] = "8237-92"

In [68]:
test.head()['defendant']

314598                 FREDERICK WILLIAMS
46340     ERIC JONES|AND|JOYCE TODD JONES
111571                      MCDOUGALL IAN
181511    DEBRA RODGERS|AND|NICOLE GRAHAM
494988     MUNN JEANNETT|AND|MUNN PATRICK
Name: defendant, dtype: object

In [93]:
first = 'FREDERICK'
last = 'WILLIAMS'
mask = (test['defendant'].str.contains(first)) & (test['defendant'].str.contains(last))
test['voter_record'] = test['voter_record'].mask(mask,"")

In [94]:
test.head()

,Unnamed: 0,id,premises_x,lat,lon,number,street,city,state,zip,...,non_residential,unaware,months,publichousing,first_hearing_dow,latitude,longitude,clean_address,firstname,voter_record
314598,421758,LT-07-05-22-6633,"1018 S 48TH ST, PHILADELPHIA, PA",39.945041,-75.215368,1018,S 48th St,Philadelphia,PA,19143,...,f,f,4.29274,f,5.0,NaN,NaN,NaN,NaN,
46340,61868,LT-03-03-06-0436,"6060 CRESCENTVILLE RD, PHILADELPHIA, PA",40.041607,-75.114752,6060,Crescentville Rd,Philadelphia,PA,19120,...,f,f,NaN,f,2.0,NaN,NaN,NaN,NaN,
111571,149409,LT-96-03-15-1419,"1324 LOCUST ST, PHILADELPHIA, PA",39.948065,-75.163382,1324,Locust St,Philadelphia,PA,19107,...,f,f,NaN,f,2.0,NaN,NaN,NaN,NaN,
181511,243153,LT-08-10-14-5155,"233 E PENN ST, PHILADELPHIA, PA",40.034680,-75.166712,233,E Penn St,Philadelphia,PA,19144,...,f,f,3.00000,f,4.0,NaN,NaN,NaN,NaN,
494988,663353,LT-95-11-07-0432,"2220 WALNUT ST, PHILADELPHIA, PA",39.951001,-75.178310,2220,Walnut St,Philadelphia,PA,19103,...,f,f,NaN,f,2.0,NaN,NaN,NaN,NaN,


In [2]:
len(matched)

NameError: name 'matched' is not defined

<h1> Adding zip code to this thing </h1>

In [163]:
voters = pd.read_csv('all_voters.csv',encoding='utf-8')

In [187]:
properties = pd.read_csv('../opa_properties_public.csv',usecols=['location','owner_1','owner_2','lat','lng','zip_code'])

In [167]:
voters = voters.rename(columns={'0':'ID'})

voters['location'] = voters['house_number'].astype(str) + " " + voters['street']

<h4> Standardize addresses </h4>

In [203]:
p = PassyunkParser()
zipcode = 19143
v_chunk = voters.loc[voters['zip']==zipcode]
v_chunk['location'] = v_chunk['location'].apply(lambda x: p.parse(x)['components']['output_address'])

C:\Users\catst\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [204]:
p_chunk = properties.loc[properties['zip_code'].str.contains(str(zipcode),na=False)]
p_chunk['location'] = p_chunk['location'].apply(lambda x: p.parse(x)['components']['output_address'])

C:\Users\catst\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


<h4> merge baby merge </h4>

In [205]:
merged = pd.merge(v_chunk,p_chunk,how='left',on='location')

In [206]:
geocoded = merged.loc[~merged['lat'].isnull()]
print "number of voters: " + str(len(v_chunk))
print "number of geocoded voters: " + str(len(geocoded))
print "success: " + str(float(len(geocoded)) / len(v_chunk) * 100) + "%"

number of voters: 43703
number of geocoded voters: 34279
success: 78.4362629568%


In [207]:
geocoded = geocoded.drop('zip_code',axis=1)

In [208]:
path="voters_geocoded_" + str(zipcode) + ".csv"
geocoded.to_csv(path,encoding='utf-8')